<a href="https://colab.research.google.com/github/ridhiaggarwal06/airline-ops-sql-analysis/blob/main/Schema%2Bqueries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. **Schema Implementation**

In [ ]:
import sqlite3

#This statement creates a connection labelled as conn.  This will be used throughout to ensure the consistency for when we start to query the database tables.
conn = sqlite3.connect('airline.db')
cursor = conn.cursor()
#ensuring that foreign keys are on for referential integrity
cursor.execute("PRAGMA foreign_keys = ON")

# Creating aircraft table
cursor.execute('''
CREATE TABLE aircrafts (
    aircraft_id VARCHAR(32) PRIMARY KEY NOT NULL,
    aircraft_model TEXT,
    aircraft_quota REAL,
    aircraft_limit REAL,
    seats_capacity INT

);
''')

# Creating airports table
cursor.execute('''
CREATE TABLE airports (
    airport_id VARCHAR(32) PRIMARY KEY NOT NULL,
    airport_name TEXT,
    airport_city TEXT,
    airport_country TEXT

);
''')

# Creating routes table
cursor.execute('''
CREATE TABLE routes (
    route_id VARCHAR(32) PRIMARY KEY NOT NULL,
    arrival_airport VARCHAR(32),
    flight_duration REAL CHECK (flight_duration > 0),
    FOREIGN KEY (arrival_airport) REFERENCES airports(airport_id)

);
''')

# Creating flights table
cursor.execute('''
CREATE TABLE flights (
    flight_id VARCHAR(32) PRIMARY KEY NOT NULL,
    route_id VARCHAR(32),
    departure_date DATETIME,
    departure_day TEXT,
    aircraft_id VARCHAR(32),
    number_of_crew INT,
    seats_booked INT,
    FOREIGN KEY (route_id) REFERENCES routes(route_id),
    FOREIGN KEY (aircraft_id) REFERENCES aircrafts(aircraft_id)


);
''')

# Creating costs table
cursor.execute('''
CREATE TABLE costs (
    cost_id VARCHAR(32) PRIMARY KEY NOT NULL,
    flight_id VARCHAR(32) UNIQUE,
    fuel_cost REAL,
    crew_salary REAL,
    turnaround_cost REAL,
    FOREIGN KEY (flight_id) REFERENCES flights(flight_id) ON DELETE SET NULL

);
''')

# Creating transactions table
cursor.execute('''
CREATE TABLE transactions (
    transaction_id VARCHAR(32) PRIMARY KEY NOT NULL,
    flight_id VARCHAR(32),
    payment_amount REAL CHECK (payment_amount > 0),
    seat_category TEXT,
    FOREIGN KEY (flight_id) REFERENCES flights(flight_id) ON DELETE SET NULL


);
''')




#This saves the chnages to the databae.  Up unitl this point the executed SQL statement isn't stored, changes are not immediatley saved.
conn.commit()

print("Database and tables created successfully!")


Database and tables created successfully!


In [ ]:
# Checking tables and their columns
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table_name in tables:
    print(f"Table: {table_name[0]}")
    cursor.execute(f"PRAGMA table_info({table_name[0]});")
    columns = cursor.fetchall()
    for col in columns:
        print(f"  Column: {col[1]}, Type: {col[2]}, NotNull: {col[3]}, DefaultVal: {col[4]}, PrimaryKey: {col[5]}")
    print("-" * 20)



Table: aircrafts
  Column: aircraft_id, Type: VARCHAR(32), NotNull: 1, DefaultVal: None, PrimaryKey: 1
  Column: aircraft_model, Type: TEXT, NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: aircraft_quota, Type: REAL, NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: aircraft_limit, Type: REAL, NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: seats_capacity, Type: INT, NotNull: 0, DefaultVal: None, PrimaryKey: 0
--------------------
Table: airports
  Column: airport_id, Type: VARCHAR(32), NotNull: 1, DefaultVal: None, PrimaryKey: 1
  Column: airport_name, Type: TEXT, NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: airport_city, Type: TEXT, NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: airport_country, Type: TEXT, NotNull: 0, DefaultVal: None, PrimaryKey: 0
--------------------
Table: routes
  Column: route_id, Type: VARCHAR(32), NotNull: 1, DefaultVal: None, PrimaryKey: 1
  Column: arrival_airport, Type: VARCHAR(32), NotNull: 0, DefaultVal: None, PrimaryKey:

In [ ]:
# Upload CSV files
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving AircraftTable.csv to AircraftTable.csv
Saving airporttable.csv to airporttable.csv
Saving finalcosttable.csv to finalcosttable.csv
Saving finalfinaltransactiontable.csv to finalfinaltransactiontable.csv
Saving finalflighttable(1).csv to finalflighttable(1).csv
Saving finalroutetable.csv to finalroutetable.csv
User uploaded file "AircraftTable.csv" with length 736 bytes
User uploaded file "airporttable.csv" with length 5012 bytes
User uploaded file "finalcosttable.csv" with length 26850 bytes
User uploaded file "finalfinaltransactiontable.csv" with length 7008897 bytes
User uploaded file "finalflighttable(1).csv" with length 41610 bytes
User uploaded file "finalroutetable.csv" with length 718 bytes


In [ ]:
# Function to import CSV into a table
import csv

def import_csv_to_table(csv_file, table_name):
    #opens the file aas read only 'r', doesn't allow the origianl csv to be changed.
    with open(csv_file, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip header row if present
        for row in csv_reader:
            #? creates a placeholder for each column in the CSV file. ['?','?','?'] - Join makes it a string so it can then be inserted.
            # use of the '?' reduce risk of SQL injection
            placeholders = ', '.join(['?' for _ in row])
            #Assumes that the CSV and table have the same structure (this could be an issue) Would have to specify column names if different.
            sql = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.execute(sql, row)

# Import data from CSV files into the relevant table. import_csv_to_table is the function, passing the two values across.
try:
    import_csv_to_table('airporttable.csv', 'airports')
    import_csv_to_table('AircraftTable.csv', 'aircrafts')
    import_csv_to_table('finalroutetable.csv', 'routes')
    import_csv_to_table('finalflighttable(1).csv', 'flights')
    import_csv_to_table('finalcosttable.csv', 'costs')
    import_csv_to_table('finalfinaltransactiontable.csv', 'transactions')
    conn.commit()
    print("Data imported successfully!")
except Exception as e:
    print(f"An error occurred: {e}")
    conn.rollback()  # Rollback changes if an error occurred


Data imported successfully!


In [ ]:
import pandas as pd

# Query all six tables and load into pandas DataFrames
flights_df = pd.read_sql_query("SELECT * FROM flights", conn)
aircrafts_df = pd.read_sql_query("SELECT * FROM aircrafts", conn)
airports_df=pd.read_sql_query("SELECT * FROM airports",conn)
routes_df=pd.read_sql_query("SELECT * FROM routes",conn)
transactions_df=pd.read_sql_query("SELECT * FROM transactions",conn)
costs_df=pd.read_sql_query("SELECT * FROM costs",conn)


# Show the first 10 lines of each DataFrame
print("\nFlights Table:")
print(flights_df.head(10))
print("\nAircrafts Table:")
print(aircrafts_df.head(10))
print("\nAirports Table:")
print(airports_df.head(10))
print("\nRoutes Table:")
print(routes_df.head(10))
print("\nTransactions Table:")
print(transactions_df.head(10))
print("\nCosts Table:")
print(costs_df.head(10))


Flights Table:
                    flight_id                    route_id departure_date  \
0  01JP7Q3EMY876TT2CCTRKJMHT8  01JP5JPNR7BJBZDRBPAWEE51C1      5/20/2024   
1  01JP7Q3ENCNX1JW8M6WF4M14P7  01JP5JPNR91N1BSRC2CRYXFS35      5/21/2024   
2  01JP7Q3ENEP1NT89JRBBDKNRRM  01JP5JPNR9RSDJSMXZRQYNCGJK      5/29/2024   
3  01JP7Q3ENGKC7RM67RTH09R1EY  01JP5JPNR61PTYNAQQTCKMFKD4      5/14/2024   
4  01JP7Q3ENJGNMSQDGBNZKKMNV7  01JP5JPNREVQ0W429NCPYMBAXN      5/14/2024   
5  01JP7Q3ENNKADTZNZ1X9H1P47B  01JP5JPNRBDQM6X7TEBYAZNGET      5/10/2024   
6  01JP7Q3ENQ69M5W62FG1R5NJX5  01JP5JPNREH90ZG9MG7F6Q3J5T      5/26/2024   
7  01JP7Q3ENS643MZ3YXFZ37BY26  01JP5JPNR9604RKK3E6WCPN0Q2      5/15/2024   
8  01JP7Q3ENW6TX2CZ7FXS73E4QY  01JP5JPNRCTJ03ATQBRWH1WH4P      5/20/2024   
9  01JP7Q3ENYS74Q3B6V055C1DJ9  01JP5JPNR9RSDJSMXZRQYNCGJK       5/5/2024   

  departure_day aircraft_id  number_of_crew  seats_booked  
0        Monday          16              15           288  
1       Tuesday          19

## 2. EDA

In [ ]:
#checking the quality of the data by ensuring no null values

print(flights_df.isnull().sum())
print(airports_df.isnull().sum())
print(aircrafts_df.isnull().sum())
print(routes_df.isnull().sum())
print(transactions_df.isnull().sum())
print(costs_df.isnull().sum())
#no null values in any of the tables

flight_id         0
route_id          0
departure_date    0
departure_day     0
aircraft_id       0
number_of_crew    0
seats_booked      0
dtype: int64
airport_id         0
airport_name       0
airport_city       0
airport_country    0
dtype: int64
aircraft_id       0
aircraft_model    0
aircraft_quota    0
aircraft_limit    0
seats_capacity    0
dtype: int64
route_id           0
arrival_airport    0
flight_duration    0
dtype: int64
transaction_id    0
flight_id         0
payment_amount    0
seat_category     0
dtype: int64
cost_id            0
flight_id          0
fuel_cost          0
crew_salary        0
turnaround_cost    0
dtype: int64


In [ ]:
#some basic statistics
totalrevenue_df=pd.read_sql_query("SELECT SUM(payment_amount) AS totalrevenue FROM transactions", conn)
print(totalrevenue_df)

totalcost_df=pd.read_sql_query("SELECT SUM(fuel_cost+crew_salary+turnaround_cost) AS totalcost FROM costs",conn)
print(totalcost_df)

totalflights_df=pd.read_sql_query("SELECT COUNT(*) AS number_of_flights FROM flights",conn)
print(totalflights_df)

totalaircrafts_df=pd.read_sql_query("SELECT COUNT(*) AS number_of_aircraft FROM aircrafts",conn)
print(totalaircrafts_df)

totalroutes_df=pd.read_sql_query("SELECT COUNT(*) AS number_of_routes FROM routes",conn)
print(totalroutes_df)

totaltransaction_df=pd.read_sql_query("SELECT COUNT(*) AS number_of_transactions FROM transactions",conn)
print(totaltransaction_df)

#total revenue is about 78 million, with costs at about 22 million over 138000 transactions
#total 40 aircraft are used over 20 routes, with 500 flights flown over the month

   totalrevenue
0    78108505.0
    totalcost
0  22606505.0
   number_of_flights
0                500
   number_of_aircraft
0                  40
   number_of_routes
0                20
   number_of_transactions
0                  138094


## 3. Data Analysis

In [ ]:
# Peak Travel days
# Objective: Identify the days of the week with the highest passenger demand.
# Method: Aggregate total seats booked per departure day and order descending

flightsdemandbyday_df = pd.read_sql_query("SELECT SUM(seats_booked), departure_day FROM flights GROUP BY departure_day ORDER BY SUM(seats_booked) DESC", conn)
print(flightsdemandbyday_df)


   SUM(seats_booked) departure_day
0              47313     Wednesday
1              42606      Thursday
2              11613       Tuesday
3              11124        Friday
4               8979      Saturday
5               8417        Sunday
6               8042        Monday


Insight - Wednesdays and Thursday have the highest bookings, followed by Tuesdays.

In [ ]:
# 2. Highest Booked Routes (Demand)
# Objective: Find the top routes with the greatest demand based on total bookings.

flightsdemand_df = pd.read_sql_query("SELECT SUM(seats_booked), route_id FROM flights GROUP BY route_id ORDER BY SUM(seats_booked) DESC", conn)
print(flightsdemand_df)

# Then, join with airports to get city names for better interpretation.
citynames_df=pd.read_sql_query("SELECT route_id, airport_city FROM airports,routes WHERE airports.airport_id=routes.arrival_airport",conn)
print(citynames_df)


    SUM(seats_booked)                    route_id
0                9990  01JP5JPNR61PTYNAQQTCKMFKD4
1                9483  01JP5JPNR6Y5KD32S8YZAAQT8R
2                9265  01JP5JPNRCP9MJ5RAHSVX45KQY
3                8846  01JP5JPNR84FSFKEX2PV0CN0C9
4                8423  01JP5JPNR7BJBZDRBPAWEE51C1
5                7607  01JP5JPNR8SKG268NAGBBE99S7
6                7356  01JP5JPNRBSS6X83A35REQ0ZMH
7                7260  01JP5JPNRDFQNQXBFA9SZHNZNJ
8                7028  01JP5JPNREH90ZG9MG7F6Q3J5T
9                6734  01JP5JPNR91N1BSRC2CRYXFS35
10               6620  01JP5JPNR9604RKK3E6WCPN0Q2
11               6613  01JP5JPNRBDQM6X7TEBYAZNGET
12               6549  01JP5JPNRCTJ03ATQBRWH1WH4P
13               6388  01JP5JPNREVQ0W429NCPYMBAXN
14               5567  01JP5JPNRCPJDJBKB17KSGDQQJ
15               5310  01JP5JPNRAP661YYAJKA4QPCFV
16               5159  01JP5JPNR72TVSKD8RCABR998H
17               4993  01JP5JPNRDNFHCETWN5KX32TKF
18               4588  01JP5JPNRAXVT13JSB0JDC18K3


Insight - Top 5 booked routes are to Tokyo, Barcelona, Paris, Turkey, and Rome.

In [ ]:
# 3. Seat Occupancy (Fill Rate)
# Objective: Calculate how full the flights are (seats booked ÷ total capacity).
# Method: Join flights with aircrafts to get seat capacity, then compute fill rate.

fillrate_df = pd.read_sql_query("SELECT route_id, SUM(seats_booked)*1.0/SUM(seats_capacity) AS FillRate FROM flights,aircrafts WHERE flights.aircraft_id = aircrafts.aircraft_id GROUP BY route_id ORDER BY FillRate DESC", conn)
print(fillrate_df)


                      route_id  FillRate
0   01JP5JPNRCP9MJ5RAHSVX45KQY  0.817596
1   01JP5JPNR84FSFKEX2PV0CN0C9  0.812827
2   01JP5JPNRDFQNQXBFA9SZHNZNJ  0.811899
3   01JP5JPNR7BJBZDRBPAWEE51C1  0.807419
4   01JP5JPNR91N1BSRC2CRYXFS35  0.807144
5   01JP5JPNR6Y5KD32S8YZAAQT8R  0.805966
6   01JP5JPNRAP661YYAJKA4QPCFV  0.804911
7   01JP5JPNRCPJDJBKB17KSGDQQJ  0.803667
8   01JP5JPNRBDQM6X7TEBYAZNGET  0.802062
9   01JP5JPNRCTJ03ATQBRWH1WH4P  0.800905
10  01JP5JPNR9RSDJSMXZRQYNCGJK  0.800111
11  01JP5JPNR72TVSKD8RCABR998H  0.798483
12  01JP5JPNR8SKG268NAGBBE99S7  0.797129
13  01JP5JPNR9604RKK3E6WCPN0Q2  0.791014
14  01JP5JPNREVQ0W429NCPYMBAXN  0.787572
15  01JP5JPNRBSS6X83A35REQ0ZMH  0.785142
16  01JP5JPNRAXVT13JSB0JDC18K3  0.782935
17  01JP5JPNREH90ZG9MG7F6Q3J5T  0.782280
18  01JP5JPNR61PTYNAQQTCKMFKD4  0.775561
19  01JP5JPNRDNFHCETWN5KX32TKF  0.714306


Insight - Paris has the highest fill rate, followed by Istanbul and Philadelphia.


In [ ]:
# 4. Route Revenue
# Objective: Measure total and average revenue for each route.
# Total revenue per route:

routerevenue_df = pd.read_sql_query("SELECT route_id, SUM(payment_amount) AS TotalPayment FROM (SELECT f.route_ID, t.payment_amount FROM transactions t, flights f, routes r WHERE t.flight_id = f.flight_id AND f.route_id = r.route_id) AS SubQuery GROUP BY route_id ORDER BY TotalPayment DESC", conn)
print(routerevenue_df)

# Average revenue per flight on each route:

averagerouterevenue_df = pd.read_sql_query("SELECT route_id, SUM(payment_amount)/COUNT(DISTINCT flight_id) AS AverageRevenue FROM (SELECT f.flight_id, f.route_id, t.payment_amount FROM transactions t, flights f, routes r WHERE t.flight_id = f.flight_id AND f.route_id = r.route_id) AS SubQuery GROUP BY route_id ORDER BY AverageRevenue DESC", conn)
print(averagerouterevenue_df)


                      route_ID  TotalPayment
0   01JP5JPNR61PTYNAQQTCKMFKD4    10384920.0
1   01JP5JPNREH90ZG9MG7F6Q3J5T     6037800.0
2   01JP5JPNRBSS6X83A35REQ0ZMH     5979850.0
3   01JP5JPNRBDQM6X7TEBYAZNGET     5950670.0
4   01JP5JPNR8SKG268NAGBBE99S7     5736645.0
5   01JP5JPNRCTJ03ATQBRWH1WH4P     4893120.0
6   01JP5JPNR72TVSKD8RCABR998H     4604765.0
7   01JP5JPNRDFQNQXBFA9SZHNZNJ     4568410.0
8   01JP5JPNRDNFHCETWN5KX32TKF     4474305.0
9   01JP5JPNR9RSDJSMXZRQYNCGJK     3872825.0
10  01JP5JPNREVQ0W429NCPYMBAXN     3580255.0
11  01JP5JPNR84FSFKEX2PV0CN0C9     3067000.0
12  01JP5JPNRAP661YYAJKA4QPCFV     2903565.0
13  01JP5JPNRCPJDJBKB17KSGDQQJ     2764110.0
14  01JP5JPNRAXVT13JSB0JDC18K3     2642850.0
15  01JP5JPNR6Y5KD32S8YZAAQT8R     1969287.5
16  01JP5JPNR7BJBZDRBPAWEE51C1     1784687.5
17  01JP5JPNRCP9MJ5RAHSVX45KQY     1224757.5
18  01JP5JPNR91N1BSRC2CRYXFS35      869985.0
19  01JP5JPNR9604RKK3E6WCPN0Q2      798697.5
                      route_id  AverageRevenue
0   01JP

Insight: Flights to Tokyo had the highest overall and average revenue.


In [ ]:
# 5. Underutilised Aircraft
# Objective: Identify aircraft that are rarely used in the month.


aircraftutility_df = pd.read_sql_query("SELECT COUNT(aircraft_id),aircraft_id FROM flights GROUP BY aircraft_id ORDER BY COUNT(aircraft_id) ASC",conn)
print(aircraftutility_df)
#we see that there are two aircraft that only fly once in the month, checking to see more about them

    COUNT(aircraft_id) aircraft_id
0                    1          20
1                    1          28
2                    6          39
3                    7          24
4                    8          13
5                    8          27
6                    9          11
7                    9          14
8                    9          16
9                    9           6
10                  10           1
11                  10           3
12                  10           8
13                  11          12
14                  11          36
15                  11          37
16                  11           7
17                  12          15
18                  12          18
19                  12          22
20                  12          26
21                  12          30
22                  12          38
23                  13          35
24                  14          10
25                  14          32
26                  14          40
27                  

Insight: Two aircraft were used only once, indicating potential inefficiency.

In [ ]:
# 6. Aircraft Details for Underutilisation
# Objective: Inspect the underutilised aircraft models for context.

underutilised_df = pd.read_sql_query("SELECT * FROM aircrafts WHERE aircraft_id='20' OR aircraft_id='28'",conn)
print(underutilised_df)


  aircraft_id aircraft_model  aircraft_quota  aircraft_limit  seats_capacity
0          20           A320            10.0            52.0             181
1          28           A380             3.0            60.0             538


Insight -
- Aircraft 20 is an A320 (short haul), Aircraft 28 is an A380 (long haul).
- Business context: A380 should be deployed on Tokyo route (high revenue).
- A320 should be assigned to Paris (highest demand/fill rate).

In [ ]:
# 7. Cost Estimation of New Deployments
# Objective: Estimate average operational cost of assigning new aircraft to routes.
# Average cost of flying an A380 to Tokyo:

tokyoA380cost_df = pd.read_sql_query("SELECT SUM(turnaround_cost+crew_salary+fuel_cost)/COUNT(*) AS Tokyo_Average_Cost FROM (SELECT * FROM costs c, flights f, aircrafts a WHERE c.flight_id = f.flight_id AND f.route_id = '01JP5JPNR61PTYNAQQTCKMFKD4' AND f.aircraft_id=a.aircraft_id AND a.aircraft_model='A380') AS SubQuery", conn)
print(tokyoA380cost_df)

# Average cost of flying an A320 to Paris:
parisA320cost_df = pd.read_sql_query("SELECT SUM(turnaround_cost+crew_salary+fuel_cost)/COUNT(*) AS Paris_Average_Cost FROM (SELECT * FROM costs c, flights f, aircrafts a WHERE c.flight_id = f.flight_id AND f.route_id = '01JP5JPNRCP9MJ5RAHSVX45KQY' AND f.aircraft_id=a.aircraft_id AND a.aircraft_model='A320') AS SubQuery", conn)
print(parisA320cost_df)


   Tokyo_Average_Cost
0       129541.818182
   Paris_Average_Cost
0              5477.5


Insight: A380 to Tokyo costs ~£130,000, A320 to Paris costs ~£5,500.

## 4. Final Insights

In conclusion, the best days to introduce new flights wouuld be on Wednesday and Thursdays since the number of seats booked on those days are the highest. We see that Paris, Rome, Tokyo, Istanbul and Barcelona are the most popular destinations in this monht. We also see that flights to Paris have the highest fill rate, i.e. the highest proportion of seats are filled, while flights to Tokyo yield the highest revenue. Hence, we should aim to have more flights to these 2 locations. We hence check our fleet, and find that an A380 (tail number 28) and an A320 (tail number 20) only fly once. Hence, we aim to use these two aircrafts, with the A320 for the shorter flight to Paris and the A380 for the longer haul flight to Tokyo. A cost analysis shows that for similar flights before, the average cost for the A380 to fly to Tokyo is 129541, and that for an A320 to Paris is 5477. The average revenue on these routes have been found to be 288470 and 42233 respectively, so there is significant margin to be made here.